# CINEMA. KEYS FOR SUCCESS

![](MoviesImages/collage1.jpg)

## 1. Introduction.

Cinema is known as the seventh art. But it is a big industry as well. According to EFE agency, cultural and creative industries represent 3% of worldwide PIB and employ 29.5 million people (source [EFE](https://www.efe.com/efe/espana/cultura/las-industrias-creativas-representan-el-3-por-ciento-del-pib-mundial-segun-la-unesco/10005-2780740)). So it is easy to understand there is a big interest in making cinema profitable and it is not difficult to find articles about what the keys for getting so are (https://phys.org/news/2017-11-explores-movie-successful.html). This is a topic that has already been analyzed with data science models, but of course there is always something else to say about it. This is what this study tries to do, going deep on cinema metrics to bring some light to this question: **what are the keys that determine revenue in a movie?** I will try to answer this question by creating a model to predict movies revenue.

The main source of information I have used in this study is **The Movie Database** ([TMDb](https://www.themoviedb.org/)). As written on their website:

*The Movie Database (TMDb) is a community built movie and TV database. Every piece of data has been added by our amazing community dating back to 2008. TMDb's strong international focus and breadth of data is largely unmatched and something we're incredibly proud of. Put simply, we live and breathe community and that's precisely what makes us different.* ([source](https://www.themoviedb.org/about))

The reasons for choosing TMDb have been:
* Allowed access to download their entire database of movies, actors and crew, with more than 400,000 movies and 1,200,000 actors and crew in the whole industry.
* Their very good support, with excellent documentation and help forums. This has been a big help on the process to download and understand the data.
I would like to thank TMDb for doing this study possible!

Additionally, I have used:

* A dataset of 5,000 movies downloaded from **data.world** (access through this [link](https://data.world/popculture/imdb-5000-movie-dataset)), whose original source is **Internet Movie Database** ([IMDb](https://www.imdb.com/)).
* **[The Numbers](https://www.the-numbers.com/)**, as help for double check revenue and budget data for specific movies.
* https://www.usinflationcalculator.com/, to get US inflation information from 1913 to 2018.

With regards to the script, it is all original (of course, I got help on [stackoverflow](https://stackoverflow.com/) and many other forums). The only exception to this has been the script in python 3 to download movies and cast/crew information using TMDb API, which I slighly adapted from the original of **galli-leo**. Thank you, galli-leo! (access through this [link](https://gist.github.com/galli-leo/6398f9128ffc20af70c6c7eedfeb0a65)). 

The whole script for this study consist of **eight jupyter notebooks in Python 3**, plus **this additional** including my final report. Notebooks 01 and 02 have been run from a linux computer and are used to download the whole information of movies and cast/crew through TMDb API. Notebooks 03 to 08 have been run from **Google Colab** and some parts of the script assume this. I recommend to run these notebooks (03 to 08) from Google Colab as part of the script is specific for Google Colab and gets the data I have used from my Google Drive account.

## 2. The data.

Data from TMDb have been downloaded using script on notebooks *01_Getting_Movies_Data* and *02_Getting_People_Data* through TMDb API (API [overview](https://www.themoviedb.org/documentation/api) and [documentation](https://developers.themoviedb.org/3/getting-started/introduction)). TMDb API is available for everyone to use; you only need a TMDb user account to request an API key. 

However, when using the API, we find some rate limiting. The limits are 40 requests every 10 seconds and are limited by IP address, not API key. So this is being burstable to 40 in a single second, or as an average of 4 requests/second. The timer resets 10 seconds from the first request within the current 10 second "bucket". This means that if the limit is triggered, then it is necessary to wait up to 9 seconds before the timer resets, but depending where we are within the 10 second window, it could be the very next second.

The API returns a json file with the whole information for every movie id and people (cast or crew) id. So we have two different sets of json files:
* One set made of 409,791 json files for movies (one json file = one movie).
* One set made of 1,197,556 json files for cast/crew (one json file = one person).

The API call can be personalized to define what information we want to get. At this initial point, I have selected a wide range of fields to download, just because they could be used in the future to make this study evolve or even for other studies (why not a recommendation system?). We have to take into account that the whole process of downloading took about 2 weeks! 

After getting the json files, it is time to import them to a jupyter notebook. In both datasets (movies and crew) I have previously run a script to check the structure of the files (in notebooks *03_Tidying_Movies_Data* for movies and *05_Tidying_People_Data* for people). This has been useful to know the nested structure of dictionaries and decide which information I would finally import. At this point, memory limit is a key driver and I have had to estimate if working in a cluster would be necessary. It has been finally not.

These are the fields I have finally imported for **movies**:

* **id** for TMDb (*int*)
* **titles**: list of alternative titles (*list of str*)
* **belongs_to_collection_id** (*int*)
* **belongs_to_collection_name** (*str*)
* **budget** (*int*) in US dollars.
* **credits cast**: list of cast with different additional features: **cast_id** (*int*), **character** (*str*), **gender** (*int*), **id** (*int*), **name** (*str*), **order** (*int*)
* **credits crew**: list of crew with different additional features: **department** (*str*), **gender** (*int*), **id** (*int*), **job** (*str*), **name** (*str*)
* **imdb_id** (*int*)
* **genres**: list of id (*int*) - name (*str*) of generes associated to every movie
* **keywords**: list of id (*int*) - name (*str*)
* **original_language** (*str*)
* **original_title** (*str*)
* **overview** (*str*)
* **popularity** (*float*): numeric value rating popularity coming from TMDb community
* **production_companies**: list of companies and additional features: **id** (*int*), **name** (*str*), **origin_country** (*str*)
* **production_countries**: list of **iso_3166_1** (*str*) and **name** (*str*) for production countries
* **release_date** (*str*)
* **release_dates**: list of dictionaries with information for every release: **iso_3166_1** (*str*), **certification** (*str*), **iso_639_1** (*str*), **release_date** (*str*), **type** (*int*) (numeric code for type of release)
* **revenue** (*int*) in US dollars.
* **reviews_page** (*int*)
* **reviews_results**: list of additional features: **author** (*str*), **content** (*str*), **id** (*int*)
* **reviews_total_pages** (*int*)
* **reviews_total_results** (*int*)
* **runtime** (*int*)
* **spoken_languages**: list of **iso_639_1** (*str*) and **name** (*str*) for spoken languages in the movie
* **status** (*str*) (released/production/planned...)
* **tagline** (*str*)
* **title** (*str*): title in English
* **vote_average** (*float*) (from TMDb users)
* **vote_count** (*int*)  (from TMDb users)

These are the fields I have finally imported for **people**:

* **id** (*int*) for TMDb
* **adult** (*bool*) coming/not coming form adult cinema
* **birthday** (*str*)
* **deathday** (*str*)
* **gender** (*int*)
* **imdb_id**  (*int*)
* **known_for_department** (*str*)
* **movie_credits_cast**: list of movies and additional features where the person worked as cast: **adult** (*bool*), **character** (*str*), **genre_ids** (*list of int*), **id** (*int*), **original_language** (*str*), **original_title** (*str*), **popularity** (*int*), **release_date** (*str*), **title** (*str*), **vote_average** (*float*), **vote_count** (*int*)
* **movie_credits_crew**: list of movies and additional features where the person worked as crew: **adult** (*bool*), **department** (*str*), **genre_ids** (*list of int*), **id** (*int*), **job** (*str*), **original_language** (*str*), **original_title** (*str*), **popularity** (*int*), **release_date** (*str*), **title** (*str*), **vote_average** (*float*), **vote_count** (*int*)
* **name** (*str*)
* **place_of_birth** (*str*)
* **popularity** (*int*): numeric value rating popularity coming from TMDb community
* **tv_credits_cast**: list of tv movies and additional features where the person worked as cast: **character** (*str*), **episode_count** (*int*), **first_air_date** (*str*), **genre_ids** (*list of int*), **id** (*int*), **name** (*str*), **origin_country** (*str*), **original_language** (*str*), **original_name** (*str*), **popularity** (*int*), **vote_average** (*float*), **vote_count** (*int*)
* **tv_credits_crew**: list of tv movies and additional features where the person worked as crew: **department** (*str*), **episode_count** (*int*), **first_air_date** (*str*), **genre_ids** (*list of int*), **id** (*int*), **job** (*str*), **name** (*str*), **origin_country** (*str*), **original_language** (*str*), **original_name** (*str*), **popularity** (*int*), **vote_average** (*float*), **vote_count** (*int*)

Although this is the set of features I have finally imported to my jupyter notebooks, not all of them have participated in the model. And this is because I have decided to exclude variables that in fact can be affected by the revenue itself, as their value is got some time after the release of the movie. **My goal is trying to analyze the variables that have an effect on the later revenue, but that can be controlled before the release**. For example: *popularity* will be for sure higher for movies with higher revenue. And *votes* are got after the release. But both features are not available when planning a movie. So those kind of features have been excluded. Similar case for *belongs to collection*: second and third part morvies are shot precisely because previous movies in the collection were a success! So I haven´t considered that feature either.

It would have been interesting to use tv information in our model. So I could have included the effect of having in our movie actors or crew who worked previously on tv. But I have had a problem here: one key feature in our analysis is realease date. I have created some features for our movies holding information of their cast and crew, **about their work up to that movie**. So it is critical to know the dates of their works. We don´t have that information for tv. We have the episode number our cast or crew worked on and we have the first air date of the tv series or tv movie, but we don´t know the real date of the episodes. So this information could be missleading and so I have excluded it.

But the **key issue I have had with the data has been about revenue**: after getting our 409,791 json files for movies I have discovered that less than 3% had revenue informed. This is one of the reasons why I have got the 5,000 movies dataset from data.world (originally from IMDb): increasing the number of films with revenue informed. This new dataset holds IMDb id. And we have it in our TMDb dataset as well. So merging has been possible. 

And the second reason for doing so is quality control: the information in TMDb is maintained by a big community. There are rules to do so, as showed in [TMDb bible](https://www.themoviedb.org/bible/movie#59f73b759251416e71000007). But there are many oportunities for errors. And some errors can produce outliers. And outliers are a problem. An easy error will be adding revenue (or budget) information in million instead of units. Or adding revenue (or budget) information in different currencies. The rule about it is *including revenue and budget information in dollars in the time when the movie was released*. But this is not always the case. Having another source of information has been useful to apply some quality control and clean up some outliers.

![](MoviesImages/collage3.jpg)

## 3. Methodology. The jupyter notebooks.

I will proceed now to explain the methodoly used by describing what every notebook does.

### 3.1 Notebooks *01_Getting_Movies_Data* and *02_Getting_People_Data*.

As commented, they include the code to download the whole TMDb database of movies and people (cast/crew). They have been run from a linux computer and the whole downloading process have taken about two weeks.

### 3.2 Notebook *03_Tidying_Movies_Data*.

This notebook, as the following up to *08_Modelling*, have been run in Google Colab. We recommend to do so to be able to run all the script without the need of adapting anything. This is because every of these notebooks include at the beginning (in section *Initial work*) the specific code of Google Colab to access to the data from my Google Drive account. They include as well some instructions to connect to every one Google Drive account, in order to copy some of the output. So a Google Drive account would be necessary for that.

After this initial section, the script performs the following steps:
1. By going through all movies json files, it checks and counts all the keys and values in first two levels of json files. It generates as output a file with summary structure, very useful in the whole process.
2. It imports the selected fields from all movies jsons files to a pandas dataframe.
3. Several of the columns of this new dataframe are made of elements that in fact are lists of new dictionaries. Working with this information in this format would be impossible. So some functions are defined to store the information from that columns in new dataframes in a format more easy to work with.

### 3.3 Notebook *04_Movies_First_Analysis*.

This notebook (to be run from Google Colab) performs the following steps:
1. Initial section for uploading basic libraries, getting access to my Google Drive account (to get the data) and getting access to everyone Google Drive account (just in case any export of outputs is needed). After that, script to import all movies json files to a pandas dataframe (explained more in detail in notebook 03) is run. 
2. Initial checks of the data show that only 3% of the rows have revenue informed. Information from IMDb will be well received.
3. *'5000 movies'* dataset from data.world - IMDb is imported from my Google Drive account to a dataframe, then cleaned up and merged with our original 400,000 movies dataset to add information of budget and revenue.
4. An initial criteria to merge revenue and budget information from TMDb and IMDb is set up and so new columns with this information are created. We will work only with rows where this newly defined revenue is greater than 0. Then the analysis of the data shows that our dataset is mainly made of US movies from last 60 years. Thus we decide to concentrate our analysis on **movies released on US before 2018 with revenue figures available** (we exclude 2018 as we understand that revenue figures take some time to stabilize). 
5. Once our movies universe have been reduce, we select the id's of these movies and the id's of all people have worked on them. In following notebooks we will exclusively import those movies/people.

### 3.4 Notebook *05_Tidying_People_Data*.

This notebook (to be run from Google Colab) performs the following steps:
1. Initial section for uploading basic libraries, getting access to my Google Drive account (to get the data) and getting access to everyone Google Drive account (just in case any export of outputs is needed). 
2. By going through all people json files, it checks and counts all the keys and values in first two levels of json files. It generates as output a file with summary structure, very useful in the whole process.
3. It imports the selected fields from all people jsons files to a pandas dataframe.
4. Several of the columns of this new dataframe are made of elements that in fact are lists of new dictionaries. Working with this information in this format would be impossible. So some functions are defined to store the information from that columns in new dataframes in a format more easy to work with.

### 3.5 Notebook *06_Exploratory_Data_Analysis*.

This notebook (to be run from Google Colab) performs the following steps:
1. Initial section for uploading basic libraries, getting access to my Google Drive account (to get the data) and getting access to everyone Google Drive account (just in case any export of outputs is needed). 
2. Using the script created in previous notebooks, information from our selected universe of movies is imported from json files to a dataframe, *'5000 movies'* dataset from IMDb is imported and merged with our dataframe and *'lists of dictionaries'* columns are treated.
3. Using the script created in previous notebooks, information from people who worked in our selected universe of movies is imported from json files to a dataframe and *'lists of dictionaries'* columns are treated.
4. Quality checks and format changes are perfomed in all our dataframes.
5. Deeper in detail analysis of revenue and budget, comparing information from TMDb and IMDb, let us apply new criteria to correct some errors in revenue and budget data. The analysis of data with regards to release date let us decide to limit our movies universe to the ones released from 1950. So our final universe of analysis is made of **movies released on US from 1950 to 2017 with revenue figures available**. Conclusions of our model will be applicable to this universe.

### 3.6 Notebook *07_Feature_Engineering*.

This notebook (to be run from Google Colab) performs the following steps:
1. Initial section for uploading basic libraries, getting access to my Google Drive account (to get the data) and getting access to everyone Google Drive account (just in case any export of outputs is needed). 
2. Using the script created in previous notebooks, information from our reduced universe of movies is imported from json files to a dataframe, *'5000 movies'* dataset from IMDb is imported and merged with our dataframe and *'lists of dictionaries'* columns are treated.
3. Using the script created in previous notebooks, information from people who worked in our reduced universe of movies is imported from json files to a dataframe and *'lists of dictionaries'* columns are treated.
4. Quality checks and format changes (explained deeper in detail in previous notebooks) are perfomed in all our dataframes.
5. Feature engineering for cast information: a model to create features for our universe of movies collecting information from cast is deployed. The key steps are:


* Clustering movies according to their budget (to split them in "big productions" and "others"). Inflation factor is cleaned up.
* Clustering actors in three groups according to their volume of work in previous different kind of movies.
* Definition of new features for movies: number of actors between the three main in the cast included in each of the previously defined three groups, and number of actors in other roles in the cast included in each of the previousle defined three groups. It is key in this step to consider only **the history of every actor up to every specific movie**.


6. Definition of other new features, collecting information from genres, original language, production countries, certifications, annual increase factor and crew. With regards to crew, an equivalent model than the one was created for cast is deployed now for crew for 13 main jobs in a movie. In this case, we create features for every job showing the proportion of members in every previously defined group. 
7. Finally, a dataframe with all new features is created and exported to csv to be the base of our model (in next notebook) and avoid all the usual initial work of importing jsons and tidying up.

### 3.7 Notebook *08_Modelling*.

This notebook (to be run from Google Colab) performs the following setps:
1. Uploading of the dataset with our new features.
2. Before modelling, it corrects errors in revenue and budget of some specific movies (with the help of IMDb and *The numbers* websites) and so avoids some likely outliers.
3. It fills null values in our features. 
4. It performs different models to be able to forecast movies revenue, but specially to get conclusions. We will look at this more in detail in the next point.

![](MoviesImages/collage2.jpg)

## 4. Conclusions.

We have basically used machine learning linear models in our approach. And this is because we are more interested in getting some conclusions from the model than in getting the best estimator. 

So these are the features (48) we have finally considered:
* Budget.
* Features (6) collecting information from cast.
* Features (26) collecting information form crew.
* Genres.
* Production countries.
* Original language (English vs non English).
* Runtime
* Certifications.
* Annual increase factor.

Considering the high volume of movies with no information of budget, instead of trying to fulfill it and assuming this could introduce significant bias in our model, we have firstly applied models to the movies for what we had budget.

To avoid the problem that working with such a skewed variable as revenue could mean, we have worked all the time with log(revenue) as our target.

We have firstly applied a Ridge linear model with all this variables and we have been able to explain 55% of variability (R2 score) of revenues. The differences observed between train and test are quite small, showing overfitting does not seem to be a problem. The same observation can be applied to all other models we have tried.

Our second trial was using a robust model, but we did not get better results.

Then we tried a lasso model, that reduce the number of features and preserves the most signifiant to our model. The number of features has then been reduced from 47 to 28, that we could say it is still quite high. But the good point is that we didn´t loose much of our R2 score: we still are able to explain 53% of variability. Very good!

Then we have analyzed the impact of budget in revenue, and we see that only budget is able to explain 45% of variability of revenue. And last, trying a model without budget, for all rows, we see we are able to explain 49% of variability.

We would say that much of the information of budget is contained in all other variables. As all other variables for themselves are able to explain almost the same percentage of variability. And all together (with budget) are not able to increase this figure much more.

I assume that we might ot been able to provide other features more difficult to get, about other aspects of movies as: is a impacting final important? A sad final? And open final? We could have got this from reviews. Still work to do on this!!

